In [7]:
import optuna
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset

In [8]:
data = pd.read_csv("Data/clean.csv")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Utilisation du device : {device}")

# Normalisation
dataset = data[['Close']].values
scaler = MinMaxScaler(feature_range=(-1, 1))
data_scaled = scaler.fit_transform(dataset)

Utilisation du device : cpu


In [9]:
# Création des séquences
def create_sequences(data, seq_length):
    xs, ys = [], []
    for i in range(len(data) - seq_length):
        xs.append(data[i:(i + seq_length)])
        ys.append(data[i + seq_length])
    return np.array(xs), np.array(ys)

SEQ_LENGTH = 60
X, y = create_sequences(data_scaled, SEQ_LENGTH)

# Conversion PyTorch
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

# Split 80/10/10
train_size = int(len(X) * 0.8)
val_size = int(len(X) * 0.1)

X_train, y_train = X_tensor[:train_size], y_tensor[:train_size]
X_val, y_val = X_tensor[train_size:train_size+val_size], y_tensor[train_size:train_size+val_size]

In [10]:
BATCH_SIZE = 64
train_loader = DataLoader(TensorDataset(X_train, y_train), shuffle=False, batch_size=BATCH_SIZE)
val_loader = DataLoader(TensorDataset(X_val, y_val), shuffle=False, batch_size=BATCH_SIZE)

In [11]:
class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout=0.2):
        super(GRUModel, self).__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # Notez l'utilisation de nn.GRU au lieu de nn.LSTM
        self.gru = nn.GRU(
            input_size, 
            hidden_size, 
            num_layers, 
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0
        )
        
        # Couche fully connected
        # Pas de *2 ici car ce n'est pas un Bi-GRU (sauf si on ajoute bidirectional=True)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        # Initialisation du hidden state uniquement (pas de c0 pour le GRU)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        # Forward propagate GRU
        out, _ = self.gru(x, h0)
        
        # On récupère la dernière étape de temps
        out = self.fc(out[:, -1, :])
        return out

In [12]:
def objective(trial):
    # --- Espace de recherche ---
    hidden_size = trial.suggest_int('hidden_size', 32, 256)
    num_layers = trial.suggest_int('num_layers', 1, 3)
    dropout = trial.suggest_float('dropout', 0.1, 0.5)
    lr = trial.suggest_float('lr', 1e-4, 1e-2, log=True)
    
    # --- Création du modèle ---
    model = GRUModel(input_size=1, hidden_size=hidden_size, num_layers=num_layers, output_size=1, dropout=dropout).to(device)
    
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr) # Adam est souvent meilleur pour GRU aussi
    
    # --- Entraînement rapide (Pruning) ---
    for epoch in range(10): # 10 époques suffisent pour voir si l'architecture est bonne
        model.train()
        for x_b, y_b in train_loader:
            x_b, y_b = x_b.to(device), y_b.to(device)
            optimizer.zero_grad()
            out = model(x_b)
            loss = criterion(out, y_b)
            loss.backward()
            optimizer.step()
            
        # Validation
        model.eval()
        val_losses = []
        with torch.no_grad():
            for x_v, y_v in val_loader:
                x_v, y_v = x_v.to(device), y_v.to(device)
                pred = model(x_v)
                val_losses.append(criterion(pred, y_v).item())
        
        avg_val_loss = np.mean(val_losses)
        
        # Signaler à Optuna pour qu'il arrête si c'est mauvais (Pruning)
        trial.report(avg_val_loss, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
            
    return avg_val_loss

print("Recherche des meilleurs hyperparamètres pour le GRU...")
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=30) # 30 essais

best_params = study.best_params
print(f"Meilleurs paramètres trouvés : {best_params}")

[I 2026-02-07 13:03:12,575] A new study created in memory with name: no-name-59abae33-9b3f-41d0-9004-65b9b0261a41


Recherche des meilleurs hyperparamètres pour le GRU...


[I 2026-02-07 13:04:25,083] Trial 0 finished with value: 0.0016900061891647056 and parameters: {'hidden_size': 187, 'num_layers': 2, 'dropout': 0.31184887193254296, 'lr': 0.0007275799514514403}. Best is trial 0 with value: 0.0016900061891647056.
[I 2026-02-07 13:05:01,868] Trial 1 finished with value: 0.014073827560059727 and parameters: {'hidden_size': 175, 'num_layers': 1, 'dropout': 0.48481988554520716, 'lr': 0.008471617370545598}. Best is trial 0 with value: 0.0016900061891647056.
[I 2026-02-07 13:06:02,817] Trial 2 finished with value: 0.053048030869103965 and parameters: {'hidden_size': 216, 'num_layers': 2, 'dropout': 0.2779406400523725, 'lr': 0.008269037020145118}. Best is trial 0 with value: 0.0016900061891647056.
[I 2026-02-07 13:07:10,789] Trial 3 finished with value: 0.003948730739648454 and parameters: {'hidden_size': 227, 'num_layers': 2, 'dropout': 0.4748680117347296, 'lr': 0.00072322741590765}. Best is trial 0 with value: 0.0016900061891647056.
[I 2026-02-07 13:07:48,71

Meilleurs paramètres trouvés : {'hidden_size': 211, 'num_layers': 2, 'dropout': 0.23754761354172282, 'lr': 0.0004632766293278801}


In [13]:
print("\nEntraînement du modèle final avec les meilleurs paramètres...")

final_model = GRUModel(
    input_size=1,
    hidden_size=best_params['hidden_size'],
    num_layers=best_params['num_layers'],
    output_size=1,
    dropout=best_params['dropout']
).to(device)

optimizer = optim.Adam(final_model.parameters(), lr=best_params['lr'])
criterion = nn.MSELoss()

# Paramètres d'entraînement complet
NUM_EPOCHS = 100
patience = 15
best_val_loss = float('inf')
trigger_times = 0

train_losses, val_losses = [], []

for epoch in range(NUM_EPOCHS):
    final_model.train()
    batch_losses = []
    for x_b, y_b in train_loader:
        x_b, y_b = x_b.to(device), y_b.to(device)
        optimizer.zero_grad()
        out = final_model(x_b)
        loss = criterion(out, y_b)
        loss.backward()
        optimizer.step()
        batch_losses.append(loss.item())
    
    train_loss = np.mean(batch_losses)
    
    final_model.eval()
    val_batch_losses = []
    with torch.no_grad():
        for x_v, y_v in val_loader:
            x_v, y_v = x_v.to(device), y_v.to(device)
            pred = final_model(x_v)
            val_batch_losses.append(criterion(pred, y_v).item())
            
    val_loss = np.mean(val_batch_losses)
    
    print(f"Epoch {epoch+1}/{NUM_EPOCHS} | Train Loss: {train_loss:.6f} | Val Loss: {val_loss:.6f}")
    
    # Early Stopping & Sauvegarde
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(final_model.state_dict(), 'best_gru_nflx.pth') # Sauvegarde sous un nom différent
        print("  -> Modèle GRU sauvegardé !")
        trigger_times = 0
    else:
        trigger_times += 1
        if trigger_times >= patience:
            print(f"Arrêt précoce à l'époque {epoch+1}")
            break

print("Terminé. Le modèle est sauvegardé sous 'best_gru_nflx.pth'")


Entraînement du modèle final avec les meilleurs paramètres...
Epoch 1/100 | Train Loss: 0.078972 | Val Loss: 0.027329
  -> Modèle GRU sauvegardé !
Epoch 2/100 | Train Loss: 0.031797 | Val Loss: 0.021063
  -> Modèle GRU sauvegardé !
Epoch 3/100 | Train Loss: 0.024040 | Val Loss: 0.016547
  -> Modèle GRU sauvegardé !
Epoch 4/100 | Train Loss: 0.018024 | Val Loss: 0.012772
  -> Modèle GRU sauvegardé !
Epoch 5/100 | Train Loss: 0.013091 | Val Loss: 0.009112
  -> Modèle GRU sauvegardé !
Epoch 6/100 | Train Loss: 0.008714 | Val Loss: 0.005966
  -> Modèle GRU sauvegardé !
Epoch 7/100 | Train Loss: 0.004986 | Val Loss: 0.003252
  -> Modèle GRU sauvegardé !
Epoch 8/100 | Train Loss: 0.002540 | Val Loss: 0.001777
  -> Modèle GRU sauvegardé !
Epoch 9/100 | Train Loss: 0.001228 | Val Loss: 0.001258
  -> Modèle GRU sauvegardé !
Epoch 10/100 | Train Loss: 0.000703 | Val Loss: 0.001181
  -> Modèle GRU sauvegardé !
Epoch 11/100 | Train Loss: 0.000644 | Val Loss: 0.001191
Epoch 12/100 | Train Loss: 0.